In [ ]:
import pandas as pd
import glob

files = sorted(glob.glob('run_ca*.csv'))
df = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

print(f"Loaded {len(files)} files, total rows = {len(df)}")
df.head()


In [ ]:
episode_stats = []
for (_, ticks, ep), grp in df.groupby(['run_id', 'num_ca_ticks', 'episode_index']):
    # steps survived = number of rows (one per timestep)
    steps = len(grp)
    # average decision time per step
    mean_time = grp['time_delta_ms'].mean()
    # compute action-switch rate
    actions = grp.sort_values('step_count')['action_taken'].tolist()
    if len(actions) > 1:
        switches = sum(1 for i in range(1, len(actions)) if actions[i] != actions[i-1])
        switch_rate = switches / (len(actions) - 1)
    else:
        switch_rate = 0.0

    episode_stats.append({
        'num_ca_ticks': ticks,
        'steps':        steps,
        'time_ms':      mean_time,
        'switch_rate':  switch_rate
    })

eps_df = pd.DataFrame(episode_stats)

# Aggregate by number of CA ticks
summary = eps_df.groupby('num_ca_ticks').agg(
    mean_steps   = ('steps',       'mean'),
    std_steps    = ('steps',       'std'),
    mean_time    = ('time_ms',     'mean'),
    std_time     = ('time_ms',     'std'),
    mean_switch  = ('switch_rate', 'mean'),
    std_switch   = ('switch_rate', 'std')
).reset_index()

In [ ]:
from matplotlib import pyplot as plt

# Steps Survived vs. CA Ticks 
plt.figure(figsize=(7.2, 4.5))
plt.errorbar(
    summary['num_ca_ticks'],
    summary['mean_steps'],
    yerr=summary['std_steps'],
    fmt='-o', capsize=4
)
plt.xlabel('Number of CA Ticks')
plt.ylabel('Avg Steps')
plt.title('Steps survived vs. Number of CA Ticks')
plt.xticks(summary['num_ca_ticks'])

for x, avg, std in zip(summary['num_ca_ticks'], summary['mean_steps'], summary['std_steps']):
    label = f"{avg:.1f}\n(±{std:.1f})"
    plt.annotate(
        label,
        (x, avg),
        textcoords="offset points",
        xytext=(0,7),
        ha='center',
        fontsize=10,
        bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.7)
    )

plt.tight_layout()
plt.show()

# Decision Time vs. CA Ticks
plt.figure(figsize=(7.2, 4.5))
plt.plot(
    summary['num_ca_ticks'],
    summary['mean_time'],
    '-s',
    label='Avg Time (ms)'
)
plt.xlabel('Number of CA Ticks')
plt.ylabel('Avg Decision Time (ms)')
plt.title('Decision Time vs. Number of CA Ticks')
plt.xticks(summary['num_ca_ticks'])

for x, avg in zip(summary['num_ca_ticks'], summary['mean_time']):
    label = f"{avg:.1f}"
    plt.annotate(
        label,
        (x, avg),
        textcoords="offset points",
        xytext=(0,7),
        ha='center',
        fontsize=10,
        bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.7)
    )

plt.tight_layout()
plt.show()


In [ ]:
# Controller Stability vs. CA Ticks
plt.figure(figsize=(6,4))
plt.bar(
    summary['num_ca_ticks'], summary['mean_switch'],
    yerr=summary['std_switch'], capsize=5, color='gray', alpha=0.7
)
plt.xlabel('Number of CA Ticks')
plt.ylabel('Avg Action-Switch Rate')
plt.xticks(summary['num_ca_ticks'])
plt.title('Controller Stability vs. Number of CA Ticks')
plt.tight_layout()
plt.show()